In [1]:
import os
import pandas as pd
from azure.storage.filedatalake import DataLakeServiceClient
from io import BytesIO

# Ladda .env OM den inte redan är laddad
if 'STORAGE_ACCOUNT_NAME' not in os.environ:
    env_file = '.env'
    if os.path.exists(env_file):
        with open(env_file, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#') and '=' in line:
                    key, value = line.split('=', 1)
                    value = value.strip().strip('"').strip("'")
                    os.environ[key.strip()] = value

import config as cfg

def read_parquet_from_adls(path):
    svc = DataLakeServiceClient(
        account_url=f"https://{cfg.STORAGE_ACCOUNT_NAME}.dfs.core.windows.net",
        credential=cfg.STORAGE_ACCOUNT_KEY
    )
    fs = svc.get_file_system_client(cfg.CONTAINER_NAME)
    fc = fs.get_file_client(path)
    data = fc.download_file().readall()
    buf = BytesIO(data)
    return pd.read_parquet(buf)

df = read_parquet_from_adls("curated/trips_combined_total.parquet")
df.head()

,AdvertisedTrainIdent,TripStartDate,LocationSignatureDeparture,LocationSignatureArrival,DepartureAdvertised,DepartureActual,ArrivalAdvertised,ArrivalActual,DelayMinutes,DurationActualMinutes,...,trip_typeoftraffic,departure_station,arrival_station,end_station_county,Deviation_Description,start_hour,start_day_of_month,start_month,is_weekday,is_delayed
0,10994,2025-10-09,Cst,U,2025-10-09 22:11:00,2025-10-09 22:11:24,2025-10-09 22:51:00,2025-10-09 22:49:17,-1.716667,38,...,YNA001,Stockholms central,Uppsala central,Uppsala län,None,22,9,10,1,0
1,1154,2025-10-09,Mc,Hb,2025-10-09 22:12:00,2025-10-09 22:12:51,2025-10-09 22:53:00,2025-10-09 22:52:44,-0.266667,40,...,YNA001,Malmö central,Helsingborgs central,Skåne län,None,22,9,10,1,0
2,1156,2025-10-09,Hie,Lu,2025-10-09 22:18:00,2025-10-09 22:18:43,2025-10-09 22:40:00,2025-10-09 22:39:13,-0.783333,20,...,YNA001,Hyllie,Lund c,Skåne län,None,22,9,10,1,0
3,1160,2025-10-09,Hie,Lu,2025-10-09 22:48:00,2025-10-09 22:48:21,2025-10-09 23:09:00,2025-10-09 23:11:39,2.650000,23,...,YNA001,Hyllie,Lund c,Skåne län,None,22,9,10,1,0
4,1162,2025-10-09,Hie,Lu,2025-10-09 23:03:00,2025-10-09 23:03:22,2025-10-09 23:22:00,2025-10-09 23:22:53,0.883333,20,...,YNA001,Hyllie,Lund c,Skåne län,None,23,9,10,1,0
